<h1 align=center><font size = 5>Capstone Project - Group Japanese Cities</font></h1>


## Introduce:In this project,I will try to group 47 Japanese major cities to several types.

## 1. Prepare the Dataset

In [264]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


There are 47 prefectures in Japan.I plan to take their 47 capital cities out and explore their common and difference.Since it's not a big deal I will just list them out instead of scraping.


In [265]:
jp_city_list = ["名古屋市","秋田市","青森市","千葉市","松山市","福井市","福岡市","福島市","岐阜市","前橋市","広島市","札幌市",\
               "神戸市","水戸市","金沢市","盛岡市","高松市","鹿児島市","横浜市","高知市","熊本市","京都市","津市","仙台市","宮崎市",\
               "長野市","長崎市","奈良市","新潟市","大分市","岡山市","那覇市","大阪市","佐賀市","さいたま市","大津市","松江市",\
               "静岡市","宇都宮市","徳島市","東京市","鳥取市","富山市","和歌山市","山形市","山口市","甲府市"]

In [267]:
city_data = pd.DataFrame({'Name':jp_city_list})
city_data.head()

,Name
0,名古屋市
1,秋田市
2,青森市
3,千葉市
4,松山市


#### Use geopy library to get the latitude and longitude values of Nagoya City.


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.


In [268]:
geolocator = Nominatim(user_agent="Japan_explorer")

city_data['Major_Dist_Coord']= city_data['Name'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))
city_data[['Latitude', 'Longitude']] = city_data['Major_Dist_Coord'].apply(pd.Series)

city_data.drop(['Major_Dist_Coord'], axis=1, inplace=True)
city_data

,Name,Latitude,Longitude
0,名古屋市,35.185105,136.899844
1,秋田市,39.719763,140.103467
2,青森市,40.822152,140.747124
3,千葉市,35.609985,140.118126
4,松山市,33.839519,132.765352
5,福井市,36.061751,136.226054
6,福岡市,33.589899,130.401751
7,福島市,37.760777,140.474581
8,岐阜市,35.423095,136.762753
9,前橋市,36.389342,139.063283


#### Define Foursquare Credentials and Version


In [269]:
CLIENT_ID = 'YMFIHHL1CFXWQKICQY5ZM505BCVM5WLFFF0IA3SVCLN4OIGI' # your Foursquare ID
CLIENT_SECRET = 'EF1LW24QYDRHBSPJVKRAXKPVTQYSPKJPH3TOTFLM2HXGFRDZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YMFIHHL1CFXWQKICQY5ZM505BCVM5WLFFF0IA3SVCLN4OIGI
CLIENT_SECRET:EF1LW24QYDRHBSPJVKRAXKPVTQYSPKJPH3TOTFLM2HXGFRDZ


#### Let's explore the first city Nagoya in our dataframe.


Get the city's name.


In [270]:
city_data.loc[0, 'Name']

'名古屋市'

Get the city's latitude and longitude values.


In [271]:
nagoya_latitude = city_data.loc[0, 'Latitude'] 
nagoya_longitude = city_data.loc[0, 'Longitude'] 

nagoya_name = city_data.loc[0, 'Name'] 

print('Latitude and longitude values of {} are {}, {}.'.format(nagoya_name, 
                                                               nagoya_latitude, 
                                                               nagoya_longitude))

Latitude and longitude values of 名古屋市 are 35.1851045, 136.8998438.


#### Now, let's get the top 500 venues that are in Nagoya within a radius of 5000 meters.


In [278]:
# type your answer here
LIMIT = 500
radius = 5000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    nagoya_latitude, 
    nagoya_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=YMFIHHL1CFXWQKICQY5ZM505BCVM5WLFFF0IA3SVCLN4OIGI&client_secret=EF1LW24QYDRHBSPJVKRAXKPVTQYSPKJPH3TOTFLM2HXGFRDZ&v=20180605&ll=35.1851045,136.8998438&radius=5000&limit=500'

Send the GET request and examine the resutls


In [279]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6039cadeb9a1cf0a8794892a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Nagoya-shi',
  'headerFullLocation': 'Nagoya-shi',
  'headerLocationGranularity': 'city',
  'totalResults': 247,
  'suggestedBounds': {'ne': {'lat': 35.230104545000046,
    'lng': 136.95480074903267},
   'sw': {'lat': 35.14010445499996, 'lng': 136.84488685096736}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5223e838498e029b92d96bd7',
       'name': '名古屋城 天守閣',
       'location': {'address': '中区本丸1-1',
        'lat': 35.18526299527807,
        'lng': 136.89990931377108,
        'labeledLatLngs': [{'label': 'display',
          'lat': 35.18526299527807,
          'lng

In [280]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a _pandas_ dataframe.


In [281]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(30)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,名古屋城 天守閣,Castle,35.185263,136.899909
1,Nagoya Castle (名古屋城),Castle,35.185555,136.899096
2,本丸御殿,Castle,35.184845,136.900329
3,名古屋能楽堂,Theater,35.182242,136.896976
4,肉うどん さんすけ,Udon Restaurant,35.176312,136.897880
5,Meijo Park (名城公園),Park,35.188736,136.902959
6,しら河 浄心本店,Unagi Restaurant,35.191671,136.891914
7,天ぷら かき揚げ 光村,Tempura Restaurant,35.188994,136.912860
8,Nagoya Kitsch et Bio (ナゴヤ キッチュ エ ビオ),Grocery Store,35.175819,136.907545
9,喫茶、食堂、民宿。 なごのや,Hostel,35.176252,136.891652


And how many venues were returned by Foursquare?


In [210]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


<a id='item2'></a>


## 2. Explore all the 47 Japanese cities


#### Let's create a function to repeat the same process to all the neighborhoods in Nagoya


In [320]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Name', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called city_venues.


In [212]:
# type your answer here
city_venues = getNearbyVenues(names=city_data['Name'],
                                   latitudes=city_data['Latitude'],
                                   longitudes=city_data['Longitude']
                                  )

名古屋市
秋田市
青森市
千葉市
松山市
福井市
福岡市
福島市
岐阜市
前橋市
広島市
札幌市
神戸市
水戸市
金沢市
盛岡市
高松市
鹿児島市
横浜市
高知市
熊本市
京都市
津市
仙台市
宮崎市
長野市
長崎市
奈良市
新潟市
大分市
岡山市
那覇市
大阪市
佐賀市
さいたま市
大津市
松江市
静岡市
宇都宮市
徳島市
東京市
鳥取市
富山市
和歌山市
山形市
山口市
甲府市


In [213]:
city_venues

,Name,Name Latitude,Name Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,名古屋市,35.185105,136.899844,名古屋城 天守閣,35.185263,136.899909,Castle
1,名古屋市,35.185105,136.899844,Nagoya Castle (名古屋城),35.185555,136.899096,Castle
2,名古屋市,35.185105,136.899844,本丸御殿,35.184845,136.900329,Castle
3,名古屋市,35.185105,136.899844,名古屋能楽堂,35.182242,136.896976,Theater
4,名古屋市,35.185105,136.899844,Main Gate (名古屋城 正門),35.183523,136.897582,Historic Site
5,名古屋市,35.185105,136.899844,Meijo Park (名城公園),35.188736,136.902959,Park
6,名古屋市,35.185105,136.899844,二之丸大手二之門,35.182863,136.900629,Historic Site
7,名古屋市,35.185105,136.899844,名古屋城 二之丸庭園,35.183955,136.902968,Garden
8,名古屋市,35.185105,136.899844,Dolphins Arena (ドルフィンズアリーナ),35.183072,136.902552,Stadium
9,名古屋市,35.185105,136.899844,名古屋城 東門,35.183959,136.903099,Historic Site


#### Let's check the size of the resulting dataframe


In [214]:
print(city_venues.shape)
city_venues.head()

(2274, 7)


,Name,Name Latitude,Name Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,名古屋市,35.185105,136.899844,名古屋城 天守閣,35.185263,136.899909,Castle
1,名古屋市,35.185105,136.899844,Nagoya Castle (名古屋城),35.185555,136.899096,Castle
2,名古屋市,35.185105,136.899844,本丸御殿,35.184845,136.900329,Castle
3,名古屋市,35.185105,136.899844,名古屋能楽堂,35.182242,136.896976,Theater
4,名古屋市,35.185105,136.899844,Main Gate (名古屋城 正門),35.183523,136.897582,Historic Site


Let's check how many venues were returned for each neighborhood


In [215]:
city_venues.groupby('Name').count()

,Name Latitude,Name Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Name,,,,,,
さいたま市,27,27,27,27,27,27
京都市,42,42,42,42,42,42
仙台市,100,100,100,100,100,100
佐賀市,35,35,35,35,35,35
前橋市,14,14,14,14,14,14
千葉市,65,65,65,65,65,65
名古屋市,24,24,24,24,24,24
和歌山市,31,31,31,31,31,31
大分市,66,66,66,66,66,66


#### Let's find out how many unique categories can be curated from all the returned venues


In [216]:
print('There are {} uniques categories.'.format(len(city_venues['Venue Category'].unique())))

There are 211 uniques categories.


<a id='item3'></a>


## 3. Analyze Each Neighborhood


In [217]:
# one hot encoding
city_onehot = pd.get_dummies(city_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
city_onehot['Name'] = city_venues['Name'] 

# move neighborhood column to the first column
fixed_columns = [city_onehot.columns[-1]] + list(city_onehot.columns[:-1])
city_onehot = city_onehot[fixed_columns]

city_onehot.head()

,Name,ATM,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Stadium,Bath House,Bavarian Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beijing Restaurant,Bistro,Boarding House,Bookstore,Boutique,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Butcher,Café,Camera Store,Canal,Candy Store,Caribbean Restaurant,Castle,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Creperie,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donburi Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Event Space,Exhibit,Fabric Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gift Shop,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Historic Site,History Museum,Hobby Shop,Hostel,Hot Spring,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Family Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Karaoke Box,Korean Restaurant,Kushikatsu Restaurant,Lake,Light Rail Station,Liquor Store,Martial Arts School,Memorial Site,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorsports Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nabe Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Optical Shop,Outdoor Sculpture,Outdoor Supply Store,Paella Restaurant,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Pet Café,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Rock Club,Sake Bar,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Soccer Stadium,Southern / Soul Food Restaurant,Souvenir Shop,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Takoyaki Place,Tea Room,Teishoku Restaurant,Tempura Restaurant,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Toll Booth,Tonkatsu Restaurant,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Tram Station,Tree,Turkish Restaurant,Udon Restaurant,Unagi Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wagashi Place,Water Park,Whisky Bar,Wine Bar,Wings Joint,Yakitori Restaurant,Yoshoku Restaurant
0,名古屋市,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,名古屋市,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,名古屋市,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

And let's examine the new dataframe size.


In [218]:
city_onehot.shape

(2274, 212)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [219]:
city_grouped = city_onehot.groupby('Name').mean().reset_index()
city_grouped

,Name,ATM,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Stadium,Bath House,Bavarian Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beijing Restaurant,Bistro,Boarding House,Bookstore,Boutique,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Butcher,Café,Camera Store,Canal,Candy Store,Caribbean Restaurant,Castle,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Creperie,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donburi Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Event Space,Exhibit,Fabric Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gift Shop,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Historic Site,History Museum,Hobby Shop,Hostel,Hot Spring,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Family Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Karaoke Box,Korean Restaurant,Kushikatsu Restaurant,Lake,Light Rail Station,Liquor Store,Martial Arts School,Memorial Site,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorsports Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nabe Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Optical Shop,Outdoor Sculpture,Outdoor Supply Store,Paella Restaurant,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Pet Café,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Rock Club,Sake Bar,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Soccer Stadium,Southern / Soul Food Restaurant,Souvenir Shop,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Takoyaki Place,Tea Room,Teishoku Restaurant,Tempura Restaurant,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Toll Booth,Tonkatsu Restaurant,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Tram Station,Tree,Turkish Restaurant,Udon Restaurant,Unagi Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wagashi Place,Water Park,Whisky Bar,Wine Bar,Wings Joint,Yakitori Restaurant,Yoshoku Restaurant
0,さいたま市,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.037037,0.000000,0.037037,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.00,0.074074,0.00,0.000000,0.000000,0.148148,0.000000,0.00000,0.000000,0.000000,0.00,0.037037,0.00,0.000000,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.037037,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037037,0.037037,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00

#### Let's confirm the new size


In [220]:
city_grouped.shape

(47, 212)

#### Let's print each neighborhood along with the top 5 most common venues


In [221]:
num_top_venues = 5

for city in city_grouped['Name']:
    print("----"+city+"----")
    temp = city_grouped[city_grouped['Name'] == city].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----秋田市----
               venue  freq
0  Convenience Store  0.15
1        Coffee Shop  0.07
2        Pizza Place  0.07
3       Noodle House  0.07
4         Steakhouse  0.07


----秋田市----
                       venue  freq
0        Japanese Restaurant  0.12
1          Convenience Store  0.10
2                      Hotel  0.10
3           Sushi Restaurant  0.05
4  Japanese Curry Restaurant  0.05


----秋田市----
                 venue  freq
0  Japanese Restaurant  0.10
1    Convenience Store  0.10
2                 Café  0.09
3     Ramen Restaurant  0.09
4            BBQ Joint  0.04


----秋田市----
                 venue  freq
0    Convenience Store  0.17
1     Ramen Restaurant  0.09
2  Japanese Restaurant  0.09
3        Train Station  0.06
4             Sake Bar  0.06


----秋田市----
                        venue  freq
0           Convenience Store  0.21
1           Outdoor Sculpture  0.14
2                        Café  0.07
3  Japanese Family Restaurant  0.07
4                 Coffee Shop  0

#### Let's put that into a _pandas_ dataframe


First, let's write a function to sort the venues in descending order.


In [244]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 5 venues for each neighborhood.


In [284]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['Name'] = city_grouped['Name']

for ind in np.arange(city_grouped.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(city_grouped.iloc[ind, :], num_top_venues)

city_venues_sorted

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,さいたま市,Convenience Store,Coffee Shop,Pizza Place,Steakhouse,Noodle House
1,京都市,Japanese Restaurant,Convenience Store,Hotel,Japanese Curry Restaurant,Sushi Restaurant
2,仙台市,Japanese Restaurant,Convenience Store,Café,Ramen Restaurant,BBQ Joint
3,佐賀市,Convenience Store,Japanese Restaurant,Ramen Restaurant,Sake Bar,Train Station
4,前橋市,Convenience Store,Outdoor Sculpture,Japanese Restaurant,Café,Bookstore
5,千葉市,Ramen Restaurant,Café,Coffee Shop,Convenience Store,Electronics Store
6,名古屋市,Historic Site,Castle,Intersection,Yoshoku Restaurant,Ramen Restaurant
7,和歌山市,Convenience Store,Ramen Restaurant,BBQ Joint,Shopping Mall,Yoshoku Restaurant
8,大分市,Convenience Store,Japanese Restaurant,Café,Hotel,BBQ Joint
9,大津市,Intersection,Convenience Store,Japanese Restaurant,Steakhouse,Train Station


<a id='item4'></a>


## 4. Cluster Neighborhoods


Run _k_-means to cluster the neighborhood into 5 clusters.


In [285]:
# set number of clusters
kclusters = 5

city_grouped_clustering = city_grouped.drop('Name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:] 

array([4, 4, 4, 4, 3, 0, 0, 2, 4, 3, 0, 4, 3, 1, 4, 0, 4, 3, 2, 4, 4, 0,
       0, 4, 4, 4, 0, 1, 4, 2, 4, 4, 0, 4, 0, 3, 2, 4, 0, 3, 1, 2, 0, 0,
       4, 4, 4])

In [286]:
city_venues_sorted

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,さいたま市,Convenience Store,Coffee Shop,Pizza Place,Steakhouse,Noodle House
1,京都市,Japanese Restaurant,Convenience Store,Hotel,Japanese Curry Restaurant,Sushi Restaurant
2,仙台市,Japanese Restaurant,Convenience Store,Café,Ramen Restaurant,BBQ Joint
3,佐賀市,Convenience Store,Japanese Restaurant,Ramen Restaurant,Sake Bar,Train Station
4,前橋市,Convenience Store,Outdoor Sculpture,Japanese Restaurant,Café,Bookstore
5,千葉市,Ramen Restaurant,Café,Coffee Shop,Convenience Store,Electronics Store
6,名古屋市,Historic Site,Castle,Intersection,Yoshoku Restaurant,Ramen Restaurant
7,和歌山市,Convenience Store,Ramen Restaurant,BBQ Joint,Shopping Mall,Yoshoku Restaurant
8,大分市,Convenience Store,Japanese Restaurant,Café,Hotel,BBQ Joint
9,大津市,Intersection,Convenience Store,Japanese Restaurant,Steakhouse,Train Station


Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [287]:
# add clustering labels
city_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

city_merged = city_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
city_merged = city_merged.join(city_venues_sorted.set_index('Name'), on='Name')

city_merged.head() # check the last columns!

,Name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,名古屋市,35.185105,136.899844,0,Historic Site,Castle,Intersection,Yoshoku Restaurant,Ramen Restaurant
1,秋田市,39.719763,140.103467,2,Convenience Store,Ramen Restaurant,BBQ Joint,Bus Station,Sake Bar
2,青森市,40.822152,140.747124,2,Convenience Store,Ramen Restaurant,Hotel,Intersection,Chinese Restaurant
3,千葉市,35.609985,140.118126,0,Ramen Restaurant,Café,Coffee Shop,Convenience Store,Electronics Store
4,松山市,33.839519,132.765352,4,Convenience Store,Café,Ramen Restaurant,Bookstore,Hotel


Finally, let's visualize the resulting clusters


In [288]:
# create map
map_clusters = folium.Map(location=[nagoya_latitude, nagoya_longitude], zoom_start=5)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(city_merged['Latitude'], city_merged['Longitude'], city_merged['Name'], city_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>


## 5. Examine Clusters


Let's see if the result corresponding to our common sense

#### Cluster 1


In [322]:
city_merged.loc[city_merged['Cluster Labels'] == 0, city_merged.columns[[0] + list(range(4, city_merged.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,名古屋市,Historic Site,Castle,Intersection,Yoshoku Restaurant,Ramen Restaurant
3,千葉市,Ramen Restaurant,Café,Coffee Shop,Convenience Store,Electronics Store
6,福岡市,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Café,Nabe Restaurant
11,札幌市,Hotel,Café,Coffee Shop,Japanese Curry Restaurant,Japanese Restaurant
12,神戸市,Steakhouse,Bakery,Café,Donburi Restaurant,Coffee Shop
14,金沢市,Japanese Restaurant,Café,Convenience Store,Sake Bar,Bed & Breakfast
16,高松市,Udon Restaurant,Convenience Store,Hotel,Japanese Restaurant,Shopping Mall
18,横浜市,Café,Baseball Stadium,Hotel,Ramen Restaurant,Convenience Store
28,新潟市,Café,Convenience Store,Park,Ramen Restaurant,Chinese Restaurant
32,大阪市,Convenience Store,Coffee Shop,Train Station,Hotel,Japanese Restaurant


It looks wired... at least 0名古屋市 and 37静岡市 which share a common feature Historic Site inicating that they may be good for tourists.They should not be grouped as the same with other industrial citied like 千葉市.

#### Cluster 2


In [323]:
city_merged.loc[city_merged['Cluster Labels'] == 1, city_merged.columns[[0] + list(range(4, city_merged.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
13,水戸市,Convenience Store,Japanese Restaurant,Hotel,Soba Restaurant,Sake Bar
24,宮崎市,Convenience Store,Hotel,Ramen Restaurant,Japanese Restaurant,Bed & Breakfast
25,長野市,Convenience Store,Hotel,Ramen Restaurant,Udon Restaurant,Intersection


Looks not bad since they all have the same 1st Most Common venue.

#### Cluster 3


In [324]:
city_merged.loc[city_merged['Cluster Labels'] == 2, city_merged.columns[[0] + list(range(4, city_merged.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,秋田市,Convenience Store,Ramen Restaurant,BBQ Joint,Bus Station,Sake Bar
2,青森市,Convenience Store,Ramen Restaurant,Hotel,Intersection,Chinese Restaurant
20,熊本市,Convenience Store,Ramen Restaurant,Park,Sake Bar,Pastry Shop
30,岡山市,Convenience Store,Ramen Restaurant,Diner,Noodle House,Bus Stop
43,和歌山市,Convenience Store,Ramen Restaurant,BBQ Joint,Shopping Mall,Yoshoku Restaurant


Looks not bad but I failed to see how they are different from cluster2.

#### Cluster 4


In [292]:
city_merged.loc[city_merged['Cluster Labels'] == 3, city_merged.columns[[0] + list(range(4, city_merged.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
7,福島市,Convenience Store,Intersection,Supermarket,Japanese Restaurant,Bus Station
8,岐阜市,Bus Stop,Convenience Store,Café,Scenic Lookout,Camera Store
9,前橋市,Convenience Store,Outdoor Sculpture,Japanese Restaurant,Café,Bookstore
26,長崎市,Café,Convenience Store,Intersection,Chinese Restaurant,Hostel
35,大津市,Intersection,Convenience Store,Japanese Restaurant,Steakhouse,Train Station
38,宇都宮市,Café,Japanese Restaurant,Intersection,Convenience Store,History Museum


#### Cluster 5


In [325]:
city_merged.loc[city_merged['Cluster Labels'] == 4, city_merged.columns[[0] + list(range(4, city_merged.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,松山市,Convenience Store,Café,Ramen Restaurant,Bookstore,Hotel
5,福井市,Sake Bar,Soba Restaurant,Convenience Store,Café,Shopping Mall
10,広島市,Okonomiyaki Restaurant,Convenience Store,Ramen Restaurant,Café,Hotel
15,盛岡市,Convenience Store,Ramen Restaurant,Noodle House,Soba Restaurant,Café
17,鹿児島市,Ramen Restaurant,Convenience Store,Japanese Restaurant,Coffee Shop,Hotel
19,高知市,Convenience Store,Noodle House,Hotel,Japanese Restaurant,Sake Bar
21,京都市,Japanese Restaurant,Convenience Store,Hotel,Japanese Curry Restaurant,Sushi Restaurant
22,津市,Convenience Store,Japanese Restaurant,Mobile Phone Shop,Discount Store,Fast Food Restaurant
23,仙台市,Japanese Restaurant,Convenience Store,Café,Ramen Restaurant,BBQ Joint
27,奈良市,Convenience Store,Ramen Restaurant,Intersection,Sushi Restaurant,Noodle House


This awful!Why the super global city Tokyo and the graceful traditional city Kyoto are grouped as the same with some country cities?
It does not make sense.

## 6.Improvment

In the previous work,I just abstracted the features using a query which will retrive all the places can be found in the database without any other considerations.For example,when comparing a traveling city and a industrial city,of course both will have reataurants and convenience stores and hotels,the difference is a traveling city will have more popularity on tour spots.So to get better features,I will try to consider this,and use section=topPicks keyord which will retrive a mix of recommendations and sortByPopularity keyword to sort the results by popularity.

Below does the same thing from the previous steps.The only difference is I will use a different query to get better features.

In [298]:
LIMIT = 500
radius = 5000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&section=topPicks&sortByPopularity=1'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    nagoya_latitude, 
    nagoya_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=YMFIHHL1CFXWQKICQY5ZM505BCVM5WLFFF0IA3SVCLN4OIGI&client_secret=EF1LW24QYDRHBSPJVKRAXKPVTQYSPKJPH3TOTFLM2HXGFRDZ&v=20180605&ll=35.1851045,136.8998438&radius=5000&limit=500&section=topPicks&sortByPopularity=1'

In [299]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&section=topPicks&sortByPopularity=1'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Name', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [301]:
# type your answer here
city_venues_upgrade = getNearbyVenues(names=city_data['Name'],
                                   latitudes=city_data['Latitude'],
                                   longitudes=city_data['Longitude']
                                  )

名古屋市
秋田市
青森市
千葉市
松山市
福井市
福岡市
福島市
岐阜市
前橋市
広島市
札幌市
神戸市
水戸市
金沢市
盛岡市
高松市
鹿児島市
横浜市
高知市
熊本市
京都市
津市
仙台市
宮崎市
長野市
長崎市
奈良市
新潟市
大分市
岡山市
那覇市
大阪市
佐賀市
さいたま市
大津市
松江市
静岡市
宇都宮市
徳島市
東京市
鳥取市
富山市
和歌山市
山形市
山口市
甲府市


In [302]:
# one hot encoding
city_onehot_upgrade = pd.get_dummies(city_venues_upgrade[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
city_onehot_upgrade['Name'] = city_venues_upgrade['Name'] 

# move neighborhood column to the first column
fixed_columns = [city_onehot_upgrade.columns[-1]] + list(city_onehot_upgrade.columns[:-1])
city_onehot_upgrade = city_onehot_upgrade[fixed_columns]

city_onehot_upgrade.head()

,Name,ATM,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Stadium,Bavarian Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beijing Restaurant,Bistro,Bookstore,Boutique,Bowling Alley,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Butcher,Café,Camera Store,Canal,Candy Store,Caribbean Restaurant,Castle,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Creperie,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Donburi Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Event Space,Exhibit,Fabric Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gift Shop,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Historic Site,History Museum,Hobby Shop,Hot Spring,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Family Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Karaoke Box,Korean Restaurant,Kushikatsu Restaurant,Lake,Liquor Store,Martial Arts School,Memorial Site,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorsports Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nabe Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Optical Shop,Outdoor Sculpture,Outdoor Supply Store,Paella Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pet Café,Pharmacy,Pizza Place,Playground,Plaza,Pub,Ramen Restaurant,Record Shop,Recording Studio,Restaurant,Rock Club,Sake Bar,Sandwich Place,Sauna / Steam Room,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Soccer Stadium,Soup Place,Southern / Soul Food Restaurant,Souvenir Shop,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Takoyaki Place,Tea Room,Teishoku Restaurant,Tempura Restaurant,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tonkatsu Restaurant,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Tree,Turkish Restaurant,Udon Restaurant,Unagi Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wagashi Place,Water Park,Wine Bar,Yakitori Restaurant,Yoshoku Restaurant
0,名古屋市,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,名古屋市,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,名古屋市,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,名古屋市,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [303]:
city_grouped_upgrade = city_onehot_upgrade.groupby('Name').mean().reset_index()
city_grouped_upgrade

,Name,ATM,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Stadium,Bavarian Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beijing Restaurant,Bistro,Bookstore,Boutique,Bowling Alley,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Butcher,Café,Camera Store,Canal,Candy Store,Caribbean Restaurant,Castle,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Creperie,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Donburi Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Event Space,Exhibit,Fabric Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gift Shop,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Historic Site,History Museum,Hobby Shop,Hot Spring,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Family Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Karaoke Box,Korean Restaurant,Kushikatsu Restaurant,Lake,Liquor Store,Martial Arts School,Memorial Site,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorsports Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nabe Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Optical Shop,Outdoor Sculpture,Outdoor Supply Store,Paella Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pet Café,Pharmacy,Pizza Place,Playground,Plaza,Pub,Ramen Restaurant,Record Shop,Recording Studio,Restaurant,Rock Club,Sake Bar,Sandwich Place,Sauna / Steam Room,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Soccer Stadium,Soup Place,Southern / Soul Food Restaurant,Souvenir Shop,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Takoyaki Place,Tea Room,Teishoku Restaurant,Tempura Restaurant,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tonkatsu Restaurant,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Tree,Turkish Restaurant,Udon Restaurant,Unagi Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wagashi Place,Water Park,Wine Bar,Yakitori Restaurant,Yoshoku Restaurant
0,さいたま市,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.055556,0.000000,0.055556,0.0,0.00000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.055556,0.0000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000

In [309]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
city_venues_sorted_upgrade = pd.DataFrame(columns=columns)
city_venues_sorted_upgrade['Name'] = city_grouped_upgrade['Name']

for ind in np.arange(city_grouped_upgrade.shape[0]):
    city_venues_sorted_upgrade.iloc[ind, 1:] = return_most_common_venues(city_grouped_upgrade.iloc[ind, :], num_top_venues)

city_venues_sorted_upgrade

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,さいたま市,Coffee Shop,Noodle House,Steakhouse,Japanese Restaurant,Thrift / Vintage Store
1,京都市,Japanese Restaurant,Sushi Restaurant,Chinese Restaurant,Café,Japanese Curry Restaurant
2,仙台市,Ramen Restaurant,Café,Japanese Restaurant,Soba Restaurant,Park
3,佐賀市,Ramen Restaurant,Japanese Restaurant,Sake Bar,Bakery,Diner
4,前橋市,Outdoor Sculpture,Japanese Restaurant,Bookstore,Coffee Shop,French Restaurant
5,千葉市,Ramen Restaurant,Electronics Store,Café,Arcade,Chinese Restaurant
6,名古屋市,Historic Site,Castle,Yoshoku Restaurant,Ramen Restaurant,Snack Place
7,和歌山市,Ramen Restaurant,BBQ Joint,Shopping Mall,Yoshoku Restaurant,Tea Room
8,大分市,Japanese Restaurant,Café,Ramen Restaurant,BBQ Joint,Sake Bar
9,大津市,Japanese Restaurant,Steakhouse,Teishoku Restaurant,Bakery,Bar


In [310]:
kclusters = 5

city_grouped_clustering_upgrade = city_grouped_upgrade.drop('Name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_grouped_clustering_upgrade)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:] 

array([2, 2, 1, 1, 2, 4, 0, 4, 1, 2, 2, 1, 2, 1, 1, 2, 1, 2, 1, 2, 1, 2,
       2, 3, 2, 1, 2, 1, 1, 4, 1, 2, 2, 2, 1, 1, 4, 1, 2, 2, 4, 4, 0, 2,
       2, 1, 1])

In [312]:
# add clustering labels
city_venues_sorted_upgrade.insert(0, 'Cluster Labels', kmeans.labels_)

city_merged_upgrade = city_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
city_merged_upgrade = city_merged_upgrade.join(city_venues_sorted_upgrade.set_index('Name'), on='Name')

city_merged_upgrade.head() # check the last columns!

,Name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,名古屋市,35.185105,136.899844,0,Historic Site,Castle,Yoshoku Restaurant,Ramen Restaurant,Snack Place
1,秋田市,39.719763,140.103467,4,Ramen Restaurant,BBQ Joint,Japanese Restaurant,Café,Paper / Office Supplies Store
2,青森市,40.822152,140.747124,4,Ramen Restaurant,Café,Convenience Store,Karaoke Box,Park
3,千葉市,35.609985,140.118126,4,Ramen Restaurant,Electronics Store,Café,Arcade,Chinese Restaurant
4,松山市,33.839519,132.765352,2,Ramen Restaurant,Bookstore,Shopping Mall,Historic Site,Café


In [326]:
# create map
map_clusters_upgrade = folium.Map(location=[nagoya_latitude, nagoya_longitude], zoom_start=5)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(city_merged_upgrade['Latitude'], city_merged_upgrade['Longitude'], city_merged_upgrade['Name'], city_merged_upgrade['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Now Let's examine the clusters again.

#### Cluster 1

In [327]:
city_merged_upgrade.loc[city_merged_upgrade['Cluster Labels'] == 0, city_merged_upgrade.columns[[0] + list(range(4, city_merged_upgrade.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,名古屋市,Historic Site,Castle,Yoshoku Restaurant,Ramen Restaurant,Snack Place
37,静岡市,Historic Site,Concert Hall,Japanese Restaurant,Department Store,Ramen Restaurant


This time,I got the two cities with historic sites grouped into the same group.Good!

#### Cluster 2

In [328]:
city_merged_upgrade.loc[city_merged_upgrade['Cluster Labels'] == 1, city_merged_upgrade.columns[[0] + list(range(4, city_merged_upgrade.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
6,福岡市,Japanese Restaurant,Sushi Restaurant,Café,Ramen Restaurant,Dessert Shop
7,福島市,Supermarket,Japanese Restaurant,Dessert Shop,Sandwich Place,Indian Restaurant
13,水戸市,Japanese Restaurant,Soba Restaurant,Ramen Restaurant,Chinese Restaurant,Nabe Restaurant
17,鹿児島市,Ramen Restaurant,Japanese Restaurant,Tonkatsu Restaurant,Café,Chinese Restaurant
22,津市,Japanese Restaurant,Mobile Phone Shop,Ramen Restaurant,Shabu-Shabu Restaurant,Optical Shop
23,仙台市,Ramen Restaurant,Café,Japanese Restaurant,Soba Restaurant,Park
24,宮崎市,Japanese Restaurant,Ramen Restaurant,Gift Shop,Sake Bar,Café
27,奈良市,Ramen Restaurant,Japanese Restaurant,Motorsports Shop,Mobile Phone Shop,Noodle House
29,大分市,Japanese Restaurant,Café,Ramen Restaurant,BBQ Joint,Sake Bar
30,岡山市,Ramen Restaurant,Noodle House,Japanese Restaurant,Bookstore,Convenience Store


This group have many restaurant in their 1st,2nd features.And many are traveling cities,not bad!

#### Cluster 3

In [329]:
city_merged_upgrade.loc[city_merged_upgrade['Cluster Labels'] == 2, city_merged_upgrade.columns[[0] + list(range(4, city_merged_upgrade.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,松山市,Ramen Restaurant,Bookstore,Shopping Mall,Historic Site,Café
5,福井市,Soba Restaurant,Shopping Mall,Café,Ramen Restaurant,Japanese Restaurant
8,岐阜市,Café,Japanese Restaurant,Udon Restaurant,Dessert Shop,Deli / Bodega
9,前橋市,Outdoor Sculpture,Japanese Restaurant,Bookstore,Coffee Shop,French Restaurant
10,広島市,Okonomiyaki Restaurant,Ramen Restaurant,Café,Memorial Site,Seafood Restaurant
11,札幌市,Café,Ramen Restaurant,Coffee Shop,BBQ Joint,Japanese Curry Restaurant
12,神戸市,Café,Steakhouse,Ramen Restaurant,Bakery,Donburi Restaurant
14,金沢市,Café,Japanese Restaurant,Department Store,Restaurant,Park
15,盛岡市,Ramen Restaurant,Noodle House,History Museum,Convention Center,Sake Bar
16,高松市,Udon Restaurant,Shopping Mall,Ramen Restaurant,BBQ Joint,Café


This group is clearly different from cluster 2,they have few restaurant in their 1st,2nd,instead have more cafe,park.

#### Cluster 4

In [332]:
city_merged_upgrade.loc[city_merged_upgrade['Cluster Labels'] == 3, city_merged_upgrade.columns[[0] + list(range(4, city_merged_upgrade.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
40,東京市,Park,Chinese Restaurant,Japanese Restaurant,BBQ Joint,Historic Site


Here I get the super city Tokyo in his own group!it make sense for Tokyo has about 50% GDP of Japan.

#### Cluster 5

In [318]:
city_merged_upgrade.loc[city_merged_upgrade['Cluster Labels'] == 4, city_merged_upgrade.columns[[0] + list(range(4, city_merged_upgrade.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,秋田市,Ramen Restaurant,BBQ Joint,Japanese Restaurant,Café,Paper / Office Supplies Store
2,青森市,Ramen Restaurant,Café,Convenience Store,Karaoke Box,Park
3,千葉市,Ramen Restaurant,Electronics Store,Café,Arcade,Chinese Restaurant
20,熊本市,Ramen Restaurant,Sake Bar,Convenience Store,Unagi Restaurant,Park
25,長野市,Ramen Restaurant,Irish Pub,Convenience Store,Korean Restaurant,BBQ Joint
43,和歌山市,Ramen Restaurant,BBQ Joint,Shopping Mall,Yoshoku Restaurant,Tea Room


This group contains the north area of Japan like 秋田市 and 青森市 and 和歌山市.They are all kind of rural area.

## 7.Conclusion

In this work, I tried two ways to abstract features to divide 47 major cities.The second approach apparently has better result than the first one.By changing the query which is originally ordered by distance from the queried coordinates to popularity order,we got distinctive features than original ones.However,there are still several questions appear in the 2nd solution.For example,there are only 名古屋市 and 静岡市 clustered into a historical group.I like Osaka,Kyoto have historical sites also.Maybe it is only the foursquare query is not giving us this recommendation instead it gives more restaurant results.I don't why,maybe the local people cares more about food than its tour spots.Anyway the algorithm does good.Maybe I can try more hyper-parameters like K set it to another one.As for features,my thought is using the key features ignorant the trival ones so I only used 5 of them maybe 10 is better.